
## Классификация изображений из датасета [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html)


## Часть 1

Требования

1. Используйте фреймворк [Pytorch](https://pytorch.org/)

2. Не используйте предобученные модели.

3. Можете загрузить готовую модель или использовать собственную архитектуру.

4. Выберите способ оценки качества предсказаний модели. Обоснуйте его.

5. Проведите обучение. Продемонстрируйте умение использовать соответствующие инструменты.

6. Оцените полученный результат.

*Не используйте инструменты принцип работы которых вам непонятен.

In [ ]:
#@title Загрузка библиотек

import time
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
#@title 1) Параметры

BATCH_SIZE = 32
NUM_EPOCHS = 40
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.001

In [ ]:
#@title 2) Работа с данными

# Преобразования для данных
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4, padding_mode='reflect'),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                         std=[0.2470, 0.2435, 0.2616])
])

# Загрузка CIFAR10 и разделение на выборки
trainset = datasets.CIFAR10(root='./data', train=True,
                            download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False,
                           download=True, transform=transform)

classes = trainset.classes
print(classes)

train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = torch.utils.data.random_split(trainset, [train_size, val_size])

# Создание DataLoader'ов
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE,
                         shuffle=True, num_workers=2)
valloader = DataLoader(valset, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=2)
testloader = DataLoader(testset, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [ ]:
#@title 3) Создание модели

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(0.2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.dropout2 = nn.Dropout(0.2)
        self.conv5 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.conv6 = nn.Conv2d(256, 256, 3, padding=1)
        self.bn6 = nn.BatchNorm2d(256)
        self.dropout3 = nn.Dropout(0.2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.bn7 = nn.BatchNorm1d(512)
        self.dropout4 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.dropout1(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool(x)
        x = self.dropout2(x)
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.pool(x)
        x = self.dropout3(x)
        x = self.flatten(x)
        x = F.relu(self.bn7(self.fc1(x)))
        x = self.dropout4(x)
        x = self.fc2(x)
        return x

net = Net()

In [ ]:
#@title 4) Оптимизатор и функция потерь

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

In [ ]:
#@title 5) Перенос модели на GPU

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")

net.to(device)

Используемое устройство: cuda:0


Net(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout(p=0.2, inplace=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout2): Dropout(p=0.2, inplace=False)
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_runni

In [ ]:
#@title 6) Обучение

best_val_acc = 0.0

for epoch in range(NUM_EPOCHS):
    start_time = time.time()

    # Обучение
    net.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

    # Валидация
    net.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for data in valloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = 100 * val_correct / val_total
    print(f'Валидационная точность для эпохи {epoch+1}: {val_acc:.2f}%')

    # Сохранение лучшей модели
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            'epoch': epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
        }, 'best_model.pth')
        print('Модель сохранена!')

    end_time = time.time()
    epoch_time = end_time - start_time
    print(f'Время эпохи {epoch+1}: {epoch_time:.2f} секунд')

print('Обучение завершено!')

[1,   200] loss: 1.814
[1,   400] loss: 1.559
[1,   600] loss: 1.396
[1,   800] loss: 1.294
[1,  1000] loss: 1.218
[1,  1200] loss: 1.149
Валидационная точность для эпохи 1: 60.42%
Модель сохранена!
Время эпохи 1: 35.22 секунд
[2,   200] loss: 1.096
[2,   400] loss: 1.048
[2,   600] loss: 1.008
[2,   800] loss: 0.976
[2,  1000] loss: 0.948
[2,  1200] loss: 0.925
Валидационная точность для эпохи 2: 70.18%
Модель сохранена!
Время эпохи 2: 36.19 секунд
[3,   200] loss: 0.907
[3,   400] loss: 0.862
[3,   600] loss: 0.870
[3,   800] loss: 0.834
[3,  1000] loss: 0.829
[3,  1200] loss: 0.816
Валидационная точность для эпохи 3: 71.62%
Модель сохранена!
Время эпохи 3: 33.96 секунд
[4,   200] loss: 0.786
[4,   400] loss: 0.765
[4,   600] loss: 0.799
[4,   800] loss: 0.784
[4,  1000] loss: 0.740
[4,  1200] loss: 0.758
Валидационная точность для эпохи 4: 76.65%
Модель сохранена!
Время эпохи 4: 35.37 секунд
[5,   200] loss: 0.734
[5,   400] loss: 0.722
[5,   600] loss: 0.699
[5,   800] loss: 0.697


In [ ]:
#@title 7) Тестирование модели

# Загрузка лучшей модели
checkpoint = torch.load('best_model.pth')
net.load_state_dict(checkpoint['model_state_dict'])
net.to(device)
net.eval()

# Общая точность
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the network on the 10000 test images: {accuracy:.2f}%')

# Точность для каждого класса
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))


Accuracy of the network on the 10000 test images: 89.99%
Accuracy of airplane : 90 %
Accuracy of automobile : 93 %
Accuracy of  bird : 82 %
Accuracy of   cat : 79 %
Accuracy of  deer : 88 %
Accuracy of   dog : 84 %
Accuracy of  frog : 95 %
Accuracy of horse : 90 %
Accuracy of  ship : 95 %
Accuracy of truck : 95 %


## Часть 2

In [ ]:
#@title Загрузка библиотек

import os
import time
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision.models import resnet50
from torchvision import transforms
from tqdm import tqdm

In [ ]:
#@title 1) Параметры

batch_size = 64
learning_rate = 1e-4
num_epochs = 10
img_size = 128
random_seed = 42

In [ ]:
random.seed(random_seed)
np.random.seed(random_seed)
os.environ["PYTHONHASHSEED"] = str(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)
torch.use_deterministic_algorithms(False)

current_seed = torch.initial_seed()
print(f"Текущий seed: {current_seed}")

Текущий seed: 42


In [ ]:
#@title 2) Работа с данными

train_transforms = transforms.Compose([
    transforms.Resize(img_size),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
test_transforms = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = datasets.CIFAR10(root='./data', train=True,
                                transform=train_transforms, download=True)

classes = train_dataset.classes
print(classes)

valid_dataset = datasets.CIFAR10(root='./data', train=False,
                                transform=test_transforms, download=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
Files already downloaded and verified


In [ ]:
#@title 3) Создание модели и перенос на GPU

class ResNet50(nn.Module):
    def __init__(self, num_classes):
        super(ResNet50, self).__init__()
        self.model = resnet50(weights="DEFAULT")
        self.in_features = self.model.fc.in_features
        self.model.fc = nn.Linear(self.in_features, num_classes)

    def forward(self, x):
        x = self.model(x)
        return x

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = ResNet50(num_classes=10).to(device)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:02<00:00, 47.9MB/s]


In [ ]:
#@title 4) Оптимизатор, функция потерь и планировщик

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss().to(device)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [ ]:
#@title 5) Обучение

best_epoch = 0
best_acc = 0.0
for epoch in range(1, num_epochs + 1):
    print("-" * 40)
    print(f"Epoch: {epoch}")

    start_time = time.time()

    # Обучение
    model.train()
    epoch_loss = 0.0
    epoch_corrects = 0
    for batch_in, batch_out in tqdm(train_loader):
        batch_in = batch_in.to(device)
        batch_out = batch_out.to(device)

        y_pred = model(batch_in)
        preds = torch.argmax(y_pred, 1)
        loss = criterion(y_pred, batch_out)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() * batch_in.size(0)
        epoch_corrects += torch.sum(preds == batch_out.data)

    epoch_loss = epoch_loss / len(train_loader.dataset)
    epoch_acc = epoch_corrects / len(train_loader.dataset)
    print(f"Train loss: {epoch_loss:.4f}")
    print(f"Train acc: {epoch_acc:.4f}")

    # Валидация
    model.eval()
    epoch_loss = 0.0
    epoch_corrects = 0
    with torch.no_grad():
        for batch_in, batch_out in tqdm(valid_loader):
            batch_in = batch_in.to(device)
            batch_out = batch_out.to(device)

            y_pred = model(batch_in)
            preds = torch.argmax(y_pred, 1)
            loss = criterion(y_pred, batch_out)

            epoch_loss += loss.item() * batch_in.size(0)
            epoch_corrects += torch.sum(preds == batch_out.data)

    epoch_loss = epoch_loss / len(valid_loader.dataset)
    epoch_acc = epoch_corrects / len(valid_loader.dataset)

    # Обновление планировщика
    scheduler.step()

    print(f"Valid loss: {epoch_loss:.4f}")
    print(f"Valid acc: {epoch_acc:.4f}")

    # Сохранение лучшей модели
    if epoch_acc >= best_acc:
        best_epoch = epoch
        best_acc = epoch_acc
        torch.save(model, "./model2.pt")

    print(f"Best epoch: {best_epoch}")
    print(f"Best acc: {best_acc:.4f}")

    end_time = time.time()
    epoch_time = end_time - start_time
    print(f'Время эпохи {epoch}: {epoch_time:.2f} секунд')

    print("-" * 40)

----------------------------------------
Epoch: 1


100%|██████████| 782/782 [03:28<00:00,  3.75it/s]


Train loss: 0.3419
Train acc: 0.8938


100%|██████████| 157/157 [00:16<00:00,  9.62it/s]


Valid loss: 0.1264
Valid acc: 0.9571
Best epoch: 1
Best acc: 0.9571
Время эпохи 1: 224.89 секунд
----------------------------------------
----------------------------------------
Epoch: 2


...

Valid loss: 0.1077
Valid acc: 0.9719
Best epoch: 8
Best acc: 0.9723
Время эпохи 9: 235.28 секунд
----------------------------------------
----------------------------------------
Epoch: 10


100%|██████████| 782/782 [03:36<00:00,  3.61it/s]


Train loss: 0.0174
Train acc: 0.9942


100%|██████████| 157/157 [00:16<00:00,  9.39it/s]

Valid loss: 0.1113
Valid acc: 0.9705
Best epoch: 8
Best acc: 0.9723
Время эпохи 10: 233.27 секунд
----------------------------------------


In [ ]:
#@title 6) Тестирование модели

# Загружаем лучшую модель
model = torch.load("./model2.pt")
model.eval()

ResNet50(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (

In [ ]:
correct = 0
total = 0

# Общая точность
with torch.no_grad():
    for images, labels in tqdm(valid_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the network on the 10000 test images: {accuracy:.2f}%')


# Точность для каждого класса
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for images, labels in tqdm(valid_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
print("-" * 40)

100%|██████████| 157/157 [00:17<00:00,  9.13it/s]


Accuracy of the network on the 10000 test images: 97.23%


100%|██████████| 157/157 [00:17<00:00,  9.14it/s]

Accuracy of airplane : 98 %
Accuracy of automobile : 98 %
Accuracy of  bird : 96 %
Accuracy of   cat : 95 %
Accuracy of  deer : 96 %
Accuracy of   dog : 95 %
Accuracy of  frog : 98 %
Accuracy of horse : 98 %
Accuracy of  ship : 98 %
Accuracy of truck : 96 %
----------------------------------------


## Вывод


**Сравнительный анализ результатов:**

- Результаты модели без предобученных весов:

 -Точность на тестовом наборе: 89.99% +- 0.2

 -Время обучения: 1353.17 сек или 22.55 мин.

- Результаты модели с использованием предобученной ResNet50:

 -Точность на тестовом наборе: 97.23% +- 0.2

 -Время обучения: 2340.63 сек или 39 мин

**Вывод:**

Модель с использованием предобученной ResNet50 показала более высокую точность на тестовом наборе. Это связано с тем, что предобученная модель уже содержит знания о визуальных особенностях, полученные на большом наборе данных ImageNet. Также она обучилась быстрее, если смотреть по эпохам - в целом, достаточно было сократить количество эпох до 5.

Кроме того, предобученная модель обладает очень хорошей общей производительностью и способна эффективно классифицировать большинство объектов из этого набора данных, что отлично видно на точности по классам.

Подытожив, можно сказать, что в некоторых случаях не стоит изобретать велосипед. Использование предобученных моделей, таких как ResNet50, может быть более эффективным и результативным, чем разработка собственной модели с нуля.